In [26]:
#Import CNN Libaries (tensorflow.keras)

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization, Dropout
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import pandas as pd
import import_ipynb
from skimage.io import imread, imshow
from skimage.transform import resize
import os
from zipfile import ZipFile


In [27]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/quadeer15sh/augmented-forest-segmentation")

Skipping, found downloaded files in "./augmented-forest-segmentation" (use force=True to force download)


In [28]:

class SegmentationClassificationModel: 
    
    def __init__(self, datasetCSVFilePath, numClasses, datasetFolderPath, X_col, y_col, img_height, img_width, img_channels):
        self.datasetCSVFilePath = datasetCSVFilePath
        self.datasetFolderPath = datasetFolderPath
        self.X_col = X_col
        self.y_col = y_col
        self.img_height = img_height
        self.img_width = img_width
        self.img_channels = img_channels
        self.numClasses = numClasses
    
    def getDataset(self):
        
        myDatasetCSV = pd.read_csv(self.datasetCSVFilePath)
        # self.myDataset = pd.DataFrame(columns= [self.X_col, self.y_col])
        self.X = np.zeros((len(myDatasetCSV), self.img_height, self.img_width, self.img_channels), dtype = np.uint8)
        self.y = np.zeros((len(myDatasetCSV), self.img_height, self.img_width, 1), dtype = bool)
        i = 0
        
        
        for index, row in myDatasetCSV.iterrows():
            
            imagePath = self.datasetFolderPath + "/images/" + row[self.X_col]
            myImage = imread(imagePath)[:,:,:self.img_channels]  
            myImage = resize(myImage, (self.img_height, self.img_width), mode='constant', preserve_range=True)
            self.X[i] = myImage
            
            # Load mask
            maskDir = os.path.join(self.datasetFolderPath, "masks")
            maskArray = np.zeros((self.img_height, self.img_width, 1), dtype=bool)
            for mask_file in os.listdir(maskDir):
                if mask_file.startswith(row[self.y_col]):
                    maskPath = os.path.join(maskDir, mask_file)
                    myMask = imread(maskPath)[:, :, 0]  # Assuming grayscale
                    myMask = np.expand_dims(resize(myMask, (self.img_height, self.img_width), mode='constant',  
                                        preserve_range=True), axis=-1)
                    maskArray = np.maximum(maskArray, myMask)
        
        self.y[i] = maskArray
        i += 1
    
    def trainTestSplit(self):
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.2)
    
    def preProcessing(self):
        
        self.X_train = self.X_train / 255.0
        self.X_test = self.X_test / 255.0
        
    def trainValSplit(self):
        
        self.X_train, X_val, self.y_train, y_val = train_test_split(self.X_train, self.y_train, test_size = 0.2)
        
        self.X_val = X_val
        self.y_val = y_val
    
    def displayClassImages(self):
        
        plt.figure(figsize=(10,10))
        for i in range(6):
            plt.subplot(5,5,i+1)
            plt.xticks([])
            plt.yticks([])
            plt.grid(False)
            plt.imshow(self.X_train[i])
            plt.xlabel(self.y_train[i])
        plt.show()
            
    def augmentDataset(self, h_flip, v_flip, rot_range, myZoom_range):
        
        image_generator = ImageDataGenerator(
            horizontal_flip = h_flip,
            vertical_flip = v_flip,
            rotation_range = rot_range,
            zoom_range = myZoom_range)
        
        self.train_generator = image_generator.flow(self.X_train, self.y_train)
        
    # def cnnModel(self, numConvolutions, numDropout, dropoutRate, isBatchNormalization):
        
    #     myNumDropout = numDropout
        
    #     self.model = models.Sequential()
    #     self.model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(self.img_height, self.img_width, self.img_channels)))
    #     self.model.add(layers.MaxPooling2D((2, 2)))
        
    #     for i in range(0, numConvolutions):
            
    #         self.model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    #         self.model.add(layers.MaxPooling2D((2, 2)))
            
    #         if(myNumDropout > 0):
                
    #             self.model.add(Dropout(dropoutRate))
    #             myNumDropout -= 1
                
    #         if(isBatchNormalization == True):
                
    #             self.model.add(BatchNormalization())
        
    #     self.model.add(layers.Flatten())
    #     self.model.add(layers.Dense(64, activation='relu'))
    #     self.model.add(layers.Dense(self.num_classes))
    
    def uNetModel(self, dropoutRate):
        
        self.inputs = tf.keras.layers.Input((self.img_height, self.img_width, self.img_channels))
        
        #Contraction path
        c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(self.inputs)
        c1 = tf.keras.layers.Dropout(dropoutRate)(c1)
        c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
        b1 = tf.keras.layers.BatchNormalization()(c1)
        r1 = tf.keras.layers.ReLU()(b1)
        p1 = tf.keras.layers.MaxPooling2D((2, 2))(r1)

        c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
        c2 = tf.keras.layers.Dropout(dropoutRate)(c2)
        c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
        b2 = tf.keras.layers.BatchNormalization()(c2)
        r2 = tf.keras.layers.ReLU()(b2)
        p2 = tf.keras.layers.MaxPooling2D((2, 2))(r2)
        
        c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
        c3 = tf.keras.layers.Dropout(dropoutRate)(c3)
        c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
        b3 = tf.keras.layers.BatchNormalization()(c3)
        r3 = tf.keras.layers.ReLU()(b3)
        p3 = tf.keras.layers.MaxPooling2D((2, 2))(r3)
        
        c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
        c4 = tf.keras.layers.Dropout(dropoutRate)(c4)
        c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
        b4 = tf.keras.layers.BatchNormalization()(c4)
        r4 = tf.keras.layers.ReLU()(b4)
        p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(r4)
        
        c5 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
        b5 = tf.keras.layers.BatchNormalization()(c5)
        r5 = tf.keras.layers.ReLU()(b5)
        c5 = tf.keras.layers.Dropout(dropoutRate)(r5)
        c5 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

        # Expansive path 
        u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
        u6 = tf.keras.layers.concatenate([u6, c4])
        u6 = tf.keras.layers.BatchNormalization()(u6)
        u6 = tf.keras.layers.ReLU()(u6)

        
        u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u6)
        u7 = tf.keras.layers.concatenate([u7, c3])
        u7 = tf.keras.layers.BatchNormalization()(u7)
        u7 = tf.keras.layers.ReLU()(u7)

        
        u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(u7)
        u8 = tf.keras.layers.concatenate([u8, c2])
        u8 = tf.keras.layers.BatchNormalization()(u8)
        u8 = tf.keras.layers.ReLU()(u8)
        
        u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(u8)
        u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
        u9 = tf.keras.layers.BatchNormalization()(u9)
        u9 = tf.keras.layers.ReLU()(u9)
        
        self.outputs = tf.keras.layers.Conv2D(self.numClasses, (1, 1), activation='sigmoid')(u9)

        
    
    def compileAndFitCNNModel(self, learningRate, numEpochs):
        
        optimizer = tf.keras.optimizers.Adam(learning_rate = learningRate)
        
        
        self.model.compile(optimizer= optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

        self.history = self.model.fit(self.train_generator, epochs=numEpochs,
                            validation_data=(self.X_val, self.y_val))
        
    def compileAndFitUNetModel(self, learningRate, numEpochs):
        
        self.model = tf.keras.Model(inputs=[self.inputs], outputs=[self.outputs])
        
        optimizer = tf.keras.optimizers.Adam(learning_rate = learningRate)
        
        self.model.compile(optimizer= optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

        self.history = self.model.fit(self.train_generator, epochs=numEpochs,
                            validation_data=(self.X_val, self.y_val)) 
    
    def showModelLoss(self):
        
        #graph of the loss function between the training data and the validiation data

        plt.figure(figsize=(15, 16))

        plt.subplot(4, 2, 2)
        plt.plot(self.history.history['loss'], label='Loss')
        plt.plot(self.history.history['val_loss'], label='val_Loss')
        plt.title('Loss')
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        
    def showModelAccuracy(self):    
        
        #graph of the accuracy between the training data and the validiation data

        plt.figure(figsize=(15, 16))

        plt.subplot(4, 2, 2)
        plt.plot(self.history.history['accuracy'], label='accuracy')
        plt.plot(self.history.history['val_accuracy'], label='val_accuracy')
        plt.title('Accuracy')
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")
        plt.legend()
    
    # def showSaliencyMap(self):

    def showTable(self):
        
        # Extract accuracy, loss, validation accuracy, and validation loss values from the history object
        history_dict = self.history.history
        history_dict['epoch'] = range(1, len(history_dict['accuracy']) + 1)

        # Convert the history dictionary to a DataFrame
        self.historyDF = pd.DataFrame(history_dict)
        
        for key in history_dict.keys():
            if key != 'epoch':
                history_dict[key] = [round(value, 3) for value in history_dict[key]]

        # Create a Matplotlib figure and axis
        fig, ax = plt.subplots(figsize=(8, 6))

        # Hide axes
        ax.axis('off')
        plt.title("Accuracy and Loss over Epochs")

        # Create the table
        ax.table(cellText=self.historyDF.values,
                 colLabels=self.historyDF.columns,
                 cellLoc='center',
                 loc='center')

        # Show the plot
        plt.show()
    